In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import random
import math
import time
import torch.nn.functional as F

from tqdm.notebook import tqdm
from torch import optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import numpy as np
from  scipy import stats
import scipy
np.random.seed(1)
torch.manual_seed(1)
random.seed(1)

In [ ]:
import os
os.chdir("/content/drive/MyDrive/MADE/Project/deap")

In [ ]:
class Args:
  def __init__(self): #(data_path, epoch, batch_siz, image_size, learning_rate, weight_deca, learning_rate, learning_rate_gamma, weight_bce, load, output_dir)
    self.data_path = "/content/drive/MyDrive/MADE/semester2/CV/contest02/data/"
    self.epochs = 2
    self.batch_size = 256
    self.lr= 3e-4
    self.weight_decay= 1e-6
    self.learning_rate=None
    self.learning_rate_gamma=None
    self.weight_bce=1
    self.load=None
    self.output_dir="/content/drive/MyDrive/MADE/Project/RACNN_models/"
    self.data_dir ="./data_preprocessed_python/"# "/content/drive/MyDrive/MADE/Project/train/physionet.org/"
args = Args()   

In [ ]:
import glob
import pickle
from collections import Counter
import scipy
from  scipy import signal
from scipy.fft import fft, fftfreq

In [ ]:
import glob
import pickle
from sklearn.preprocessing import MinMaxScaler

data = []
labels = []
data_dir = './data_preprocessed_python'
files = glob.glob(os.path.join(data_dir, "*.dat"))
data_raw = []
for file_data in files:
    raw_data = pickle.load(open(file_data, 'rb'), encoding='latin1')
    data.append(raw_data['data'])
    labels.append(raw_data['labels'])

In [ ]:
labels_bin = []
for sub in range(32):
  temp = labels[sub] >= 4.5
  labels_bin.append(temp)
  #print(sum(labels_bin[sub][:, type_emotion]), end=' ')

In [ ]:
LEN_RECORD_IN_SECONDS = 60
NVIDEOS = 40
HCANALS = 9
WCANALS = 9
NTIMES_IN_SAMPLE = 128
NTIMES_IN_SEC = 128
NCANALS = 32
NFEATURES = 32
NSUBJECTS = 32
electrode_matrix = {}
electrode_matrix['FP1'] = [0, 3]
electrode_matrix['FP2'] = [0, 5]
electrode_matrix['AF3'] = [1, 3]
electrode_matrix['AF4'] = [1, 5]
electrode_matrix['F7']  = [2, 0]
electrode_matrix['F3']  = [2, 2]
electrode_matrix['FZ']  = [2, 4]
electrode_matrix['F4']  = [2, 6]
electrode_matrix['F8']  = [2, 8]
electrode_matrix['FC5']  = [3, 1]
electrode_matrix['FC1']  = [3, 3]
electrode_matrix['FC2']  = [3, 5]
electrode_matrix['FC6']  = [3, 7]
electrode_matrix['T7']  = [4, 0]
electrode_matrix['C3']  = [4, 2]
electrode_matrix['CZ']  = [4, 4]
electrode_matrix['C4']  = [4, 6]
electrode_matrix['T8']  = [4, 8]
electrode_matrix['CP5']  = [5, 1]
electrode_matrix['CP1']  = [5, 3]
electrode_matrix['CP2']  = [5, 5]
electrode_matrix['CP6']  = [5, 7]
electrode_matrix['P7']  = [6, 0]
electrode_matrix['P3']  = [6, 2]
electrode_matrix['PZ']  = [6, 4]
electrode_matrix['P4']  = [6, 6]
electrode_matrix['P8']  = [6, 8]
electrode_matrix['PO3'] = [7, 3]
electrode_matrix['PO4'] = [7, 5]
electrode_matrix['O1'] = [8, 3]
electrode_matrix['OZ'] = [8, 4]
electrode_matrix['O2'] = [8, 5]

list_electrodes = ['FP1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3',	'T7',	'CP5',	'CP1',	'P3',	'P7',	'PO3',	'O1',	'OZ',	'PZ',	'FP2',	'AF4', 'FZ', 'F4', 'F8', 'FC6',	'FC2',	'CZ', 'C4', 'T8', 'CP6',	'CP2',	'P4', 	'P8',	'PO4',	'O2']
data_dir = './data_preprocessed_python'
TRAIN_SIZE = 0.9
THRESHOLD = 4.5

In [ ]:
class EmotionDataset(Dataset):
    def __init__ (self, data, labels_bin, transforms, interval, shift, seq, type):  #indexes - индексы видео которые вошли в датасет, data - данные labels - метки бинарные
       self.data_samples = []
       self.labels = []
       self.transforms = transforms
       self.cnt = [Counter(), Counter(), Counter(),Counter()]
       for sub in range(len(data)): #sub   - человек
          for nvideo in range(NVIDEOS):
             for nsec in range(0, LEN_RECORD_IN_SECONDS - interval, shift):
                    self.data_samples.append(data[sub][nvideo, :,  (3 + nsec) * NTIMES_IN_SEC : (3 + nsec + interval) * NTIMES_IN_SEC])
                    self.labels.append(labels_bin[sub][nvideo, :])
       n = len(self.data_samples)
       #seq = np.arange(n)
       #random.shuffle(seq)
       size = int(koeff * n)
       #print(n)
       if type == 'train':
            self.data_samples = np.array(self.data_samples)[np.array(seq)][0 : size]
            self.labels = np.array(self.labels)[np.array(seq)][0 : size]
       else:
             self.data_samples = np.array(self.data_samples)[seq][size : ]
             self.labels = np.array(self.labels)[seq][size : ]

    def __len__(self):
       result =  len(self.data_samples)
       return result

    def __getitem__(self, item):
       sample = {}   
       sample = {"labels": self.labels[item],
                 "data": self.data_samples[item]
       }

       if self.transforms is not None:
           for t in self.transforms:
                sample = t(sample)
       #print(sample)         
       return sample
                 


In [ ]:
# class Compose(object):
#     def __init__(self, transforms):
#         super(Compose, self).__init__()
#         self.transforms = transforms

#     def __call__(self, sample):
#         for t in self.transforms:
#             sample = t(sample)
#         return sample

 


class to_pcc_matrix(object):
    def __init__(self):
        super(to_pcc_matrix, self).__init__()
        

    def __call__(self, sample: dict):
        canal_data = sample['data'].copy()
        list_canals = np.arange(NCANALS)
        random.shuffle(list_canals)
        canal_data_new = np.array([canal_data[list_canals[i]] for i in range(32)])
        pcc_matrix = np.corrcoef(canal_data_new)
        # for i in range(NCANALS):
        #    for j in range(i + 1):
        #       pcc_matrix[i, j] = pcc_matrix[i, j]/(np.std(canal_data_new[i]) * np.std(canal_data_new[j]))
        #       pcc_matrix[j, i] = pcc_matrix[i, j]
        sample['data'] = pcc_matrix
        #print(pcc_matrix)
        return(sample)

class ToTensor(object):

    def __init__(self, ):
        super(ToTensor, self).__init__()

    def __call__(self, sample: dict):
        return {"labels": torch.tensor(sample["labels"], dtype=torch.long),
                "data": torch.tensor(sample["data"], dtype=torch.float32),
                } 


transforms = [to_pcc_matrix(),
                   ToTensor()]                       

In [ ]:
def get_conv_module(n_input_channels,n_output_channels):
  conv = nn.Conv2d(n_input_channels, n_output_channels, kernel_size = 3, stride=1, padding='same')
  maxpool = nn.MaxPool2d(kernel_size = 2)
  bn = nn.BatchNorm2d(n_output_channels)
  relu = nn.ReLU()
  result = torch.nn.Sequential(conv, maxpool, bn, relu)
  return(result)

In [ ]:
class EmotionNet(torch.nn.Module): 
   def __init__(self, hcanals, wcanals, tures, ntimes_in_sample):
      super().__init__()
      #print("1")
      n_input_channels = [1, 32, 64, 128, ]
      n_output_channels = [32, 64, 128, 256]
      self.convs = nn.ModuleList([get_conv_module(n_input_channels[i], n_output_channels[i]) for i in range(3)])
      self.conv = nn.Conv2d(n_input_channels[3], n_output_channels[3], kernel_size = 3, stride=1, padding='same')
      self.flat = nn.Flatten(1, 3)
      self.fc = nn.Linear(4096, 2)

      
   def forward(self, input):
      #print(f"input_shape = {input.shape}")
      input = input.unsqueeze(3)
      input = input.permute(0, 3, 1, 2)
      #input (bs, h=32, w=32, 1)

      output = input
      for i, conv in enumerate(self.convs):
            output = conv(output)
            #print(outputs[i])
      output = self.conv(output)
      output = self.flat(output)
      #output (bs, s * 256,  h=9, w=9)
      output = self.fc(output)
      return output



In [ ]:
interval = 3
shift = 3
koeff = 0.8
#inds = np.arange(72960)
inds = np.arange(24320)
random.shuffle(inds)
train_dataset = EmotionDataset(data, labels_bin, transforms, interval, shift, inds, 'train')
train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
                               pin_memory=True, shuffle=True, drop_last=True)


val_dataset = EmotionDataset(data, labels_bin, transforms, interval, shift, inds, 'test')
val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
                               pin_memory=True, shuffle=False, drop_last=False)

In [ ]:
sample = val_dataset.__getitem__(300)
print(sample['data'])


tensor([[ 1.0000,  0.3272, -0.2465,  ...,  0.4084, -0.2176,  0.3754],
        [ 0.3272,  1.0000,  0.4080,  ..., -0.2889,  0.3318, -0.1702],
        [-0.2465,  0.4080,  1.0000,  ..., -0.7450,  0.6704, -0.5162],
        ...,
        [ 0.4084, -0.2889, -0.7450,  ...,  1.0000, -0.3733,  0.6521],
        [-0.2176,  0.3318,  0.6704,  ..., -0.3733,  1.0000, -0.5112],
        [ 0.3754, -0.1702, -0.5162,  ...,  0.6521, -0.5112,  1.0000]])


In [ ]:
def train(model, loader, criterion, optimizer, device, batch = None):
    model.train()
    train_loss = []
    inputs = []
   
    #lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)#, mode='min', factor=0.1, patience=10, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
    for batch in tqdm(loader, total=len(loader), desc="training...", position=0 , leave = True):

            optimizer.zero_grad()
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, 0]

            #print(batch)
            #print(trg.shape)
            levels_pred = model(src)  # B x (2 * NUM_PTS)
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()

            #usual cross entropy
            #output = levels_pred[:, 1:].reshape(-1, levels_pred.shape[-1])
            #trg1 = trg[:, 1:].reshape(-1)
            loss = criterion(levels_pred, trg) 

            #print("after")
            train_loss.append(loss.item())
            loss.backward()
            optimizer.step()
            #break
    return np.mean(train_loss)#, mid_outputs

In [ ]:
def evaluate(model, loader, criterion, device):
    
    model.eval()
    epoch_loss = 0
    history = []
  
    with torch.no_grad():
    
        for s, batch in enumerate(tqdm(loader, total=len(loader), desc="validating...", position=0 , leave = True)):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, 0]



            levels_pred = model(src)  # B x (2 * NUM_PTS)
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()

            
            loss = criterion(levels_pred, trg) 

            epoch_loss += loss.item() 
        
    return epoch_loss / s
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

In [ ]:
def calculate_predictions(model, loader):
    model.eval()
    epoch_loss = 0
    history = []
    real = []
    pred = []
    with torch.no_grad():

        for i, batch in enumerate(tqdm(loader, total=len(loader), desc="predicting...", position=0 , leave = True)):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, 0]
           

            levels_pred = model(src)  # B x (2 * NUM_PTS)
            levels_pred = levels_pred.cpu()
            #print(levels_pred.shape)
            trg_pred = levels_pred.argmax(1)
            
            real.extend(trg)
            pred.extend(trg_pred) 

            
        print(accuracy_score(real, pred)) 
        print(confusion_matrix(real, pred))  
        print(classification_report(real, pred))   
        #plt.hist(real)

In [ ]:
def get_model():
  model = EmotionNet(HCANALS, WCANALS, NFEATURES, NTIMES_IN_SAMPLE).to(device)
  return model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = get_model()
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

EmotionNet(
  (convs): ModuleList(
    (0): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
  )
  (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding

In [ ]:
criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
#optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
optimizer = optim.Adam(model.parameters(), lr=1e-4)#, momentum = 0.9)#, weight_decay=args.weight_decay)
# print(train_dataset.cnt)
# print(val_dataset.cnt)
# print(files[ind_train])
# print(files[ind_val])

In [ ]:
args.epochs = 20000
#criterion =  fnn.mse_loss
train_loss_min = 10000
val_loss_min = 10000
#batch = next(iter(train_dataloader))
for epoch in range(args.epochs):
    #logger.info(f"Starting epoch {epoch + 1}/{args.epochs}.")
    
    train_loss = train(model, train_dataloader, criterion, optimizer ,device)
    #if epoch % 500 == 0:
    print(train_loss)

    if (train_loss < train_loss_min):
        train_loss_min      = train_loss
        torch.save({
                         'model_state_dict': model.state_dict(),
                         'optimizer_state_dict': optimizer.state_dict(),
                       },
                       os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", "train.tgz")
            )  

    val_loss = evaluate(model, val_dataloader, criterion, device)
    # #break
    print(val_loss)
    #break

    # #calculate_predictions(model, val_dataloader)
    if (val_loss < val_loss_min):
         val_loss_min      = val_loss
         torch.save({'model_state_dict': model.state_dict(),    'optimizer_state_dict': optimizer.state_dict(),}, os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", f"val.tgz"))

training...:   0%|          | 0/76 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


0.7021038493043498


validating...:   0%|          | 0/19 [00:00<?, ?it/s]

0.7522205975320604


...

0.6552097813079232


validating...:   0%|          | 0/19 [00:00<?, ?it/s]

0.7342852221594917


training...:   0%|          | 0/76 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
val_loss = evaluate(model, val_dataloader, criterion, device)
# #break
print(val_loss)

In [ ]:
calculate_predictions(model, val_dataloader)